**支持度**：某一项集的记录在数据集所占的比例  
**可信度**：某一项集包含的细分项集的比例  
如何找到支持度大于0.8的所有项集？


In [17]:
def loaddata():
    return [[1,3,4],[2,3,5],[1,2,3,5],[2,5]]
def createc1(dataset):
    c1 = []
    for tran in dataset:
        for item in tran:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #print(c1)
    return list(map(frozenset,c1))#返回frozenset的列表
def scand(D,Ck,sup):
    cnt={}
    for tid in D:
        for can in Ck:
            if can.issubset(tid):#issubset()判断can是否是tid的子集
                if not can in cnt: cnt[can]=1
                else: cnt[can] += 1
    numitems = float(len(D))
    retlist = []
    supdata = {}
    for key,value in cnt.items():
        support = value/numitems
        if support>=sup:
            retlist.insert(0,key)
        supdata[key]=support
    return retlist,supdata

In [18]:
data = loaddata()
C1 = createc1(data)
D = list(map(set,data))
L1,subdata = scand(D,C1,0.5)

**组织完整的Apriori算法**

In [30]:
def genc(Lk,k):#create Ck Lk=Lk-1
    retlist = []
    lenlk = len(Lk)
    for i in range(lenlk):
        for j in range(i+1,lenlk):
            L1=list(Lk[i])[:k-2]# start from k=2 L1=[] k=3 L1= [one element]
            L2=list(Lk[j])[:k-2]#  if the first just(k-2) is same, then | 
            L1.sort()
            L2.sort()
            if L1==L2:
                retlist.append(Lk[i] | Lk[j])
    return retlist
def apriori(data,minsup=0.5):
    C1 = createc1(data)
    D = list(map(set,data))
    L1,supdata = scand(D,C1,minsup)
    L = [L1]# a list that L[0] = L1
    k=2
    while (len(L[k-2])>0):#while Ck=[] break
        Ck = genc(L[k-2],k)#L is list of Lk
        print(Ck)
        Lk,supK = scand(D,Ck,minsup)
        supdata.update(supK)# put dict supK into supdata
        L.append(Lk)
        print(L)
        k +=1
        print(k)
    return L,supdata

In [47]:
L,supdata = apriori(data,minsup=0.5)

[frozenset({2, 5}), frozenset({3, 5}), frozenset({1, 5}), frozenset({2, 3}), frozenset({1, 2}), frozenset({1, 3})]
[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})]]
3
[frozenset({2, 3, 5})]
[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})], [frozenset({2, 3, 5})]]
4
[]
[[frozenset({5}), frozenset({2}), frozenset({3}), frozenset({1})], [frozenset({2, 3}), frozenset({3, 5}), frozenset({2, 5}), frozenset({1, 3})], [frozenset({2, 3, 5})], []]
5


**关联规则生成函数**  
从一个频繁项开始，首先创建规则右部只有一个元素的规则列表进行测试，接下来合并所有剩余规则来创建一个新的规则列表，规则右部包含两个元素

In [51]:
def genru(L,supdata,minconf=0.7):
    rulist = []
    for i in range(1,len(L)):
        for freqset in L[i]:# for exam:freqset=frozenset({2, 5})
            H1 = [frozenset([item]) for item in freqset]
            print(H1)
            if (i>1):# more than two element
                ruconseq(freqset,H1,supdata,rulist,minconf)
            else:
                calcconf(freqset,H1,supdata,rulist,minconf)#freqset=frozenset({2, 3, 5})
    return rulist
def calcconf(freqset,H,supdata,br1,minconf=0.7):#for exam:freqset=frozenset({2, 5}) H=[frozenset({2}),frozenset({5})]
    pru=[]
    for conseq in H:
        conf = supdata[freqset]/supdata[freqset-conseq]#frozenset({2, 5})/frozenset({2, 5})-H[0] or frozenset({2, 5})-H[1]
        if conf >= minconf:
            print(freqset-conseq,'--->',conseq,'conf',conf)#frozenset({2, 5})-frozenset({2})--->frozenset({2})
            br1.append((freqset-conseq,conseq,conf))# record the revelance
            pru.append(conseq)
    return pru# for judge to stop cycle
def ruconseq(freqset,H,supdata,br1,minconf=0.7):
    m = len(H[0])#H = [frozenset({2}),frozenset({3}),frozenset({5})]
    if len(freqset) > (m+1):
        hmp1 = genc(H,m+1)
        hmp1 = calcconf(freqset,hmp1,supdata,br1,minconf)
        if len(hmp1) > 1:#cycle when hmp1 is [] break 
            ruconseq(freqset,hmp1,supdata,br1,minconf)
            

In [52]:
relus = genru(L,supdata,minconf=0.7)

[frozenset({2}), frozenset({3})]
[frozenset({3}), frozenset({5})]
[frozenset({2}), frozenset({5})]
frozenset({5}) ---> frozenset({2}) conf 1.0
frozenset({2}) ---> frozenset({5}) conf 1.0
[frozenset({1}), frozenset({3})]
frozenset({1}) ---> frozenset({3}) conf 1.0
[frozenset({2}), frozenset({3}), frozenset({5})]


In [53]:
relus

[(frozenset({5}), frozenset({2}), 1.0),
 (frozenset({2}), frozenset({5}), 1.0),
 (frozenset({1}), frozenset({3}), 1.0)]

**处理投票记录并创建一个交易数据库**  
这里仅是实践了代码思想

In [ ]:
#根据billnum获取两个list actionidlist titlelist
from time import sleep
from votesmart import votesmart
votesmart.apikey = '123'
def getids():
    idlist=[]
    titlelist=[]
    fr = open('recent20bills.txt')
    for line in fr.readlines():
        billnum = int(line.split('\t')[0])
        try:
            billde = votesmart.votes.getBill(billnum)#get detail of billnum
            for action in billde.actions:
                if action.level == 'House' and (action.stage=='Passage' or action.stage=='Amendment Vote'):
                    actionid = int(action.actionId)
                    print('bill: %d has actionId: %d'%(billnum,actionId))
                    idlist.append(actionid)
                    titlelist.append(line.strip().split('\t')[1])
        except:
            print('problem getting bill %d' % billnum)
        sleep(1)
    return idlist,titlelist

In [ ]:
def gettranlist(idlist,titlelist):
    meaning = ['Republican','Democratic']
    for billtitle in titlelist:
        meaning.append('%s -- Nay' %billtitle)
        meaning.append('%s -- Yea' %billtitle)
    trandict = {}
    votecount = 2
    for actionid in idlist:
        sleep(3)
        print('getting votes for actionid: %d' %actionid)
        try:
            votelist = votesmart.votes.getBillActionVotes(actionid)
            for vote in votelist:
                if not vote.candidateName in trandict:
                    trandict[vote.candidateName] = []
                    if vote.officeParties == 'Democratic':
                        trandict[vote.candidateName].append(1)
                    elif vote.officeParties == 'Republican':
                        trandict[vote.candidateName].append(0)
                if vote.action == 'Nay':
                    trandict[vote.candidateName].append(votecount)
                elif vote.action == 'Yea':
                    trandict[vote.candidateName].append(votecount+1)
        except:
            print('problem getting actionid: %d' % actionid)
        votecount += 2
    return trandict,meaning#trandict包含：每个政客对应一个列表，列表第一个对应正当，后面对应所投的票                   

In [ ]:
#上述建立事务数据库较为巧妙的是构建了trandict与meaning对应的两个列表
for item in trandict[trandict.keys()[6]]:
    print(meaning[item])#通过索引可以打印出该政客的投票详情

In [ ]:
L,sudata = apriori(data,minsup=0.5)
L

In [58]:
filename = '/Users/enjlife/machine-learning/machinelearninginaction/ch11/mushroom.dat'

In [59]:
mushdata = [line.split() for line in open(filename).readlines()]

In [60]:
L,supdata = apriori(mushdata,minsup=0.3)

[frozenset({'58', '56'}), frozenset({'116', '58'}), frozenset({'6', '58'}), frozenset({'110', '58'}), frozenset({'58', '94'}), frozenset({'58', '53'}), frozenset({'28', '58'}), frozenset({'24', '58'}), frozenset({'58', '10'}), frozenset({'58', '39'}), frozenset({'58', '2'}), frozenset({'93', '58'}), frozenset({'58', '90'}), frozenset({'58', '9'}), frozenset({'58', '86'}), frozenset({'58', '85'}), frozenset({'76', '58'}), frozenset({'58', '67'}), frozenset({'63', '58'}), frozenset({'58', '59'}), frozenset({'58', '52'}), frozenset({'58', '38'}), frozenset({'58', '36'}), frozenset({'58', '34'}), frozenset({'58', '3'}), frozenset({'58', '23'}), frozenset({'58', '1'}), frozenset({'116', '56'}), frozenset({'6', '56'}), frozenset({'110', '56'}), frozenset({'56', '94'}), frozenset({'53', '56'}), frozenset({'28', '56'}), frozenset({'24', '56'}), frozenset({'10', '56'}), frozenset({'39', '56'}), frozenset({'2', '56'}), frozenset({'93', '56'}), frozenset({'90', '56'}), frozenset({'9', '56'}), fro

[[frozenset({'58'}), frozenset({'56'}), frozenset({'116'}), frozenset({'6'}), frozenset({'110'}), frozenset({'94'}), frozenset({'53'}), frozenset({'28'}), frozenset({'24'}), frozenset({'10'}), frozenset({'39'}), frozenset({'2'}), frozenset({'93'}), frozenset({'90'}), frozenset({'9'}), frozenset({'86'}), frozenset({'85'}), frozenset({'76'}), frozenset({'67'}), frozenset({'63'}), frozenset({'59'}), frozenset({'52'}), frozenset({'38'}), frozenset({'36'}), frozenset({'34'}), frozenset({'3'}), frozenset({'23'}), frozenset({'1'})], [frozenset({'58', '85'}), frozenset({'24', '1'}), frozenset({'53', '36'}), frozenset({'116', '36'}), frozenset({'36', '56'}), frozenset({'39', '56'}), frozenset({'110', '24'}), frozenset({'110', '53'}), frozenset({'116', '34'}), frozenset({'116', '85'}), frozenset({'116', '86'}), frozenset({'116', '90'}), frozenset({'34', '56'}), frozenset({'56', '85'}), frozenset({'86', '56'}), frozenset({'90', '56'}), frozenset({'116', '56'}), frozenset({'6', '36'}), frozenset({

[[frozenset({'58'}), frozenset({'56'}), frozenset({'116'}), frozenset({'6'}), frozenset({'110'}), frozenset({'94'}), frozenset({'53'}), frozenset({'28'}), frozenset({'24'}), frozenset({'10'}), frozenset({'39'}), frozenset({'2'}), frozenset({'93'}), frozenset({'90'}), frozenset({'9'}), frozenset({'86'}), frozenset({'85'}), frozenset({'76'}), frozenset({'67'}), frozenset({'63'}), frozenset({'59'}), frozenset({'52'}), frozenset({'38'}), frozenset({'36'}), frozenset({'34'}), frozenset({'3'}), frozenset({'23'}), frozenset({'1'})], [frozenset({'58', '85'}), frozenset({'24', '1'}), frozenset({'53', '36'}), frozenset({'116', '36'}), frozenset({'36', '56'}), frozenset({'39', '56'}), frozenset({'110', '24'}), frozenset({'110', '53'}), frozenset({'116', '34'}), frozenset({'116', '85'}), frozenset({'116', '86'}), frozenset({'116', '90'}), frozenset({'34', '56'}), frozenset({'56', '85'}), frozenset({'86', '56'}), frozenset({'90', '56'}), frozenset({'116', '56'}), frozenset({'6', '36'}), frozenset({

[[frozenset({'58'}), frozenset({'56'}), frozenset({'116'}), frozenset({'6'}), frozenset({'110'}), frozenset({'94'}), frozenset({'53'}), frozenset({'28'}), frozenset({'24'}), frozenset({'10'}), frozenset({'39'}), frozenset({'2'}), frozenset({'93'}), frozenset({'90'}), frozenset({'9'}), frozenset({'86'}), frozenset({'85'}), frozenset({'76'}), frozenset({'67'}), frozenset({'63'}), frozenset({'59'}), frozenset({'52'}), frozenset({'38'}), frozenset({'36'}), frozenset({'34'}), frozenset({'3'}), frozenset({'23'}), frozenset({'1'})], [frozenset({'58', '85'}), frozenset({'24', '1'}), frozenset({'53', '36'}), frozenset({'116', '36'}), frozenset({'36', '56'}), frozenset({'39', '56'}), frozenset({'110', '24'}), frozenset({'110', '53'}), frozenset({'116', '34'}), frozenset({'116', '85'}), frozenset({'116', '86'}), frozenset({'116', '90'}), frozenset({'34', '56'}), frozenset({'56', '85'}), frozenset({'86', '56'}), frozenset({'90', '56'}), frozenset({'116', '56'}), frozenset({'6', '36'}), frozenset({

[frozenset({'110', '36', '1', '90', '24'}), frozenset({'110', '34', '1', '36', '24'}), frozenset({'110', '34', '1', '90', '24'}), frozenset({'110', '34', '1', '24', '86'}), frozenset({'110', '1', '36', '24', '86'}), frozenset({'110', '1', '90', '24', '86'}), frozenset({'110', '34', '1', '24', '85'}), frozenset({'110', '1', '36', '24', '85'}), frozenset({'110', '1', '24', '85', '86'}), frozenset({'110', '1', '90', '24', '85'}), frozenset({'36', '1', '90', '24', '85'}), frozenset({'34', '1', '36', '24', '85'}), frozenset({'34', '1', '90', '24', '85'}), frozenset({'34', '1', '53', '24', '85'}), frozenset({'34', '90', '1', '36', '24'}), frozenset({'34', '90', '1', '53', '24'}), frozenset({'34', '1', '36', '24', '86'}), frozenset({'34', '1', '90', '24', '86'}), frozenset({'34', '1', '53', '24', '86'}), frozenset({'1', '36', '24', '85', '86'}), frozenset({'34', '1', '24', '85', '86'}), frozenset({'1', '90', '24', '85', '86'}), frozenset({'53', '1', '24', '85', '86'}), frozenset({'1', '90', '

[[frozenset({'58'}), frozenset({'56'}), frozenset({'116'}), frozenset({'6'}), frozenset({'110'}), frozenset({'94'}), frozenset({'53'}), frozenset({'28'}), frozenset({'24'}), frozenset({'10'}), frozenset({'39'}), frozenset({'2'}), frozenset({'93'}), frozenset({'90'}), frozenset({'9'}), frozenset({'86'}), frozenset({'85'}), frozenset({'76'}), frozenset({'67'}), frozenset({'63'}), frozenset({'59'}), frozenset({'52'}), frozenset({'38'}), frozenset({'36'}), frozenset({'34'}), frozenset({'3'}), frozenset({'23'}), frozenset({'1'})], [frozenset({'58', '85'}), frozenset({'24', '1'}), frozenset({'53', '36'}), frozenset({'116', '36'}), frozenset({'36', '56'}), frozenset({'39', '56'}), frozenset({'110', '24'}), frozenset({'110', '53'}), frozenset({'116', '34'}), frozenset({'116', '85'}), frozenset({'116', '86'}), frozenset({'116', '90'}), frozenset({'34', '56'}), frozenset({'56', '85'}), frozenset({'86', '56'}), frozenset({'90', '56'}), frozenset({'116', '56'}), frozenset({'6', '36'}), frozenset({

[frozenset({'110', '36', '1', '24', '85', '86'}), frozenset({'110', '34', '36', '1', '24', '86'}), frozenset({'110', '34', '36', '1', '24', '85'}), frozenset({'110', '34', '36', '1', '90', '24'}), frozenset({'110', '90', '1', '36', '24', '86'}), frozenset({'110', '90', '1', '36', '24', '85'}), frozenset({'110', '90', '1', '24', '85', '86'}), frozenset({'110', '34', '1', '24', '85', '86'}), frozenset({'110', '34', '90', '1', '24', '86'}), frozenset({'110', '34', '90', '1', '24', '85'}), frozenset({'36', '90', '1', '24', '85', '86'}), frozenset({'34', '90', '1', '24', '85', '86'}), frozenset({'34', '36', '1', '24', '85', '86'}), frozenset({'34', '36', '90', '1', '24', '86'}), frozenset({'34', '36', '90', '1', '24', '85'}), frozenset({'110', '34', '36', '53', '85', '86'}), frozenset({'110', '34', '36', '53', '90', '86'}), frozenset({'110', '34', '36', '53', '90', '85'}), frozenset({'110', '53', '90', '36', '85', '86'}), frozenset({'34', '36', '53', '59', '85', '86'}), frozenset({'34', '36

[[frozenset({'58'}), frozenset({'56'}), frozenset({'116'}), frozenset({'6'}), frozenset({'110'}), frozenset({'94'}), frozenset({'53'}), frozenset({'28'}), frozenset({'24'}), frozenset({'10'}), frozenset({'39'}), frozenset({'2'}), frozenset({'93'}), frozenset({'90'}), frozenset({'9'}), frozenset({'86'}), frozenset({'85'}), frozenset({'76'}), frozenset({'67'}), frozenset({'63'}), frozenset({'59'}), frozenset({'52'}), frozenset({'38'}), frozenset({'36'}), frozenset({'34'}), frozenset({'3'}), frozenset({'23'}), frozenset({'1'})], [frozenset({'58', '85'}), frozenset({'24', '1'}), frozenset({'53', '36'}), frozenset({'116', '36'}), frozenset({'36', '56'}), frozenset({'39', '56'}), frozenset({'110', '24'}), frozenset({'110', '53'}), frozenset({'116', '34'}), frozenset({'116', '85'}), frozenset({'116', '86'}), frozenset({'116', '90'}), frozenset({'34', '56'}), frozenset({'56', '85'}), frozenset({'86', '56'}), frozenset({'90', '56'}), frozenset({'116', '56'}), frozenset({'6', '36'}), frozenset({

[[frozenset({'58'}), frozenset({'56'}), frozenset({'116'}), frozenset({'6'}), frozenset({'110'}), frozenset({'94'}), frozenset({'53'}), frozenset({'28'}), frozenset({'24'}), frozenset({'10'}), frozenset({'39'}), frozenset({'2'}), frozenset({'93'}), frozenset({'90'}), frozenset({'9'}), frozenset({'86'}), frozenset({'85'}), frozenset({'76'}), frozenset({'67'}), frozenset({'63'}), frozenset({'59'}), frozenset({'52'}), frozenset({'38'}), frozenset({'36'}), frozenset({'34'}), frozenset({'3'}), frozenset({'23'}), frozenset({'1'})], [frozenset({'58', '85'}), frozenset({'24', '1'}), frozenset({'53', '36'}), frozenset({'116', '36'}), frozenset({'36', '56'}), frozenset({'39', '56'}), frozenset({'110', '24'}), frozenset({'110', '53'}), frozenset({'116', '34'}), frozenset({'116', '85'}), frozenset({'116', '86'}), frozenset({'116', '90'}), frozenset({'34', '56'}), frozenset({'56', '85'}), frozenset({'86', '56'}), frozenset({'90', '56'}), frozenset({'116', '56'}), frozenset({'6', '36'}), frozenset({

In [62]:
for item in L[3]:
    if item.intersection('2'): print(item)

frozenset({'59', '2', '28', '34'})
frozenset({'59', '2', '63', '28'})
frozenset({'59', '2', '85', '28'})
frozenset({'59', '2', '28', '90'})
frozenset({'59', '2', '28', '39'})
frozenset({'2', '63', '28', '34'})
frozenset({'2', '85', '28', '34'})
frozenset({'2', '63', '85', '28'})
frozenset({'2', '28', '86', '34'})
frozenset({'59', '2', '28', '86'})
frozenset({'2', '63', '28', '86'})
frozenset({'2', '85', '28', '86'})
frozenset({'2', '28', '86', '90'})
frozenset({'2', '28', '86', '39'})
frozenset({'2', '28', '86', '53'})
frozenset({'2', '28', '34', '90'})
frozenset({'2', '85', '28', '90'})
frozenset({'34', '2', '28', '39'})
frozenset({'2', '63', '28', '39'})
frozenset({'2', '85', '28', '39'})
frozenset({'2', '28', '39', '90'})
frozenset({'34', '2', '28', '53'})
frozenset({'2', '85', '28', '53'})
frozenset({'2', '28', '53', '90'})
frozenset({'2', '39', '28', '53'})
frozenset({'2', '39', '34', '53'})
frozenset({'2', '85', '34', '53'})
frozenset({'2', '90', '34', '53'})
frozenset({'2', '39'